# 대스타

4차 모델 구현 CNN

### Library

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

import zipfile
import gdown

import torchvision
from torchvision import transforms

### 변수 설정

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001
training_epochs = 10
batch_size = 64

### Data Processing

In [4]:
google_path = 'https://drive.google.com/uc?id='
file_id = '1PwuXmL0HdmjsThBHDKhaCfKK3jvdoAIG'
output_name = 'train.zip'
gdown.download(google_path+file_id,output_name)

Downloading...
From: https://drive.google.com/uc?id=1PwuXmL0HdmjsThBHDKhaCfKK3jvdoAIG
To: /content/train.zip
100%|██████████| 1.37G/1.37G [00:10<00:00, 125MB/s]


'train.zip'

In [5]:
input_path = './train.zip'
output_path = './data'

In [6]:
zip_data = zipfile.ZipFile(input_path)
zip_data.extractall(output_path)

In [7]:
train_path = './data/output2/train'
val_path = './data/output2/val'
test_path = './data/output2/test'

In [8]:
trainset = torchvision.datasets.ImageFolder(root = train_path, transform=transforms.ToTensor())
valset = torchvision.datasets.ImageFolder(root = val_path, transform=transforms.ToTensor())
testset = torchvision.datasets.ImageFolder(root = test_path, transform=transforms.ToTensor())

In [9]:
print(f'Number of training examples: {len(trainset)}')
print(f'Number of validation examples: {len(valset)}')
print(f'Number of testing examples: {len(testset)}')

Number of training examples: 34461
Number of validation examples: 4207
Number of testing examples: 4564


In [10]:
data_loader = torch.utils.data.DataLoader(dataset=trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

val_loader = torch.utils.data.DataLoader(dataset=valset,
                                          batch_size=batch_size)

test_loader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=batch_size)

### 모델 구현

In [11]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 256, 256, 3)
        #    Conv     -> (?, 256, 256, 32)
        #    Pool     -> (?, 128, 128, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 128, 128, 32)
        #    Conv      ->(?, 128, 128, 64)
        #    Pool      ->(?, 64, 64, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 64x64x64 inputs -> 309 outputs
        self.fc = torch.nn.Linear(64 * 64 * 64, 309, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [12]:
# CNN 모델 정의
model = CNN().to(device)

In [13]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 538


### Training

In [15]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 3.36170077
[Epoch:    2] cost = 0.520744324
[Epoch:    3] cost = 0.252074063
[Epoch:    4] cost = 0.191820636
[Epoch:    5] cost = 0.14198488
[Epoch:    6] cost = 0.120318778
[Epoch:    7] cost = 0.0940553769
[Epoch:    8] cost = 0.077297464
[Epoch:    9] cost = 0.0577243157
[Epoch:   10] cost = 0.115856886


### val test

In [16]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)
        correct_prediction = torch.argmax(outputs, 1) == y
        accuracy = correct_prediction.float().mean()
        print(f"Accuracy: {accuracy.item()}")
        print("--------------------")

Accuracy: 0.9375
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.84375
--------------------
Accuracy: 0.84375
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.9375
--------------------
Accuracy: 0.84375
--------------------
Accuracy: 0.984375
--------------------
Accuracy: 0.859375
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.921875
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.984375
--------------------
Accuracy: 0.921875
--------------------
Accuracy: 0.921875
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.921875
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.84375
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.953125
--------------------
Accuracy: 0.953125
--------------------
Accuracy: 0.984375
--------------------
Accuracy: 0.953125
--------------

### test

In [23]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)
        correct_prediction = torch.argmax(outputs, 1) == y
        accuracy = correct_prediction.float().mean()
        print(f"Accuracy: {accuracy.item()}")
        print("--------------------")

Accuracy: 0.96875
--------------------
Accuracy: 0.90625
--------------------
Accuracy: 0.8125
--------------------
Accuracy: 0.84375
--------------------
Accuracy: 0.953125
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.953125
--------------------
Accuracy: 0.953125
--------------------
Accuracy: 0.921875
--------------------
Accuracy: 0.828125
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.875
--------------------
Accuracy: 0.96875
--------------------
Accuracy: 0.9375
--------------------
Accuracy: 1.0
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.9375
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.859375
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.953125
--------------------
Accuracy: 0.953125
--------------------
Accuracy: 0.890625
--------------------
Accuracy: 1.0
--------------------
Accuracy: 0.984375
--------------------
Accuracy: 0.9375
--------------------
Accura